In [1]:
using StatisticalRethinking, CmdStan
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.033, 0.034, 0.034, 0.032) seconds, 0.13 seconds total
Sampling took (0.065, 0.052, 0.054, 0.054) seconds, 0.22 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -891  2.3e-02    0.95  -893  -891  -891  1.8e+03  7.9e+03  1.0e+00
accept_stat__   0.93  1.3e-03   0.088  0.75  0.96   1.0  4.6e+03  2.1e+04  1.0e+00
stepsize__      0.76  3.2e-02   0.045  0.69  0.77  0.81  2.0e+00  8.9e+00  4.8e+13
treedepth__      2.1  1.5e-02    0.77   1.0   2.0   4.0  2.7e+03  1.2e+04  1.0e+00
n_leapfrog__     5.6  1.1e-01     6.2   1.0   3.0    15  3.0e+03  1.4e+04  1.0e+00
divergent__     0.00      nan    0.00  0.00  0.00  0.00      nan      nan      nan
energy__         892  3.4e-02     1.4   891   892   895  1.6e+03  7.2e+03  1.0e+00
sigma            7.8  5.9e-03    0.29   7.3   7.

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean    │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64 │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼─────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ mu         │ 154.603 │ 0.41454  │ 0.00655446 │ 0.00695315 │ 4000.0 │
│ 2   │ sigma      │ 7.76901 │ 0.289832 │ 0.00458265 │ 0.00537514 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%   │ 50.0%   │ 75.0%   │ 97.5%   │
│     │ Symbol     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ mu         │ 153.789 │ 154.326 │ 154.6   │ 154.879 │ 155.403 │
│ 2   │ sigma      │ 7.22069 │ 7.5659  │ 7.76234 │ 7.9603  │ 8.36037 │


end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*